In [1033]:
import requests as rq
import json
import pandas as pd
from pandas import DataFrame as df
import datetime
import csv
import yfinance as yf
import numpy as np
import vectorbt as vbt
import finlab_crypto
finlab_crypto.setup()
ohlcv_4h = finlab_crypto.crawler.get_all_binance('BTCUSDT', '4h')
ohlcv=finlab_crypto.crawler.get_all_binance('BTCUSDT', '1m')
# ohlcv_5m=finlab_crypto.crawler.get_all_binance('BTCUSDT', '5m')


import talib
rsi6=talib.RSI(ohlcv_4h.close,timeperiod=6)
rsi12=talib.RSI(ohlcv_4h.close,timeperiod=12)
rsi24=talib.RSI(ohlcv_4h.close,timeperiod=24)
rsi48=talib.RSI(ohlcv_4h.close,timeperiod=48)
rsi96=talib.RSI(ohlcv_4h.close,timeperiod=96)

All caught up..!
All caught up..!


In [ ]:
def get_url(bitcoin_address, limit=5000):
    base_url = "https://blockchain.info/rawaddr/"
    transactions = []
    offset = 0
    api_url = f"{base_url}{bitcoin_address}?limit={limit}&offset={offset}"
    return api_url
def get_data(address):
    timeout=99
    try:

        x=get_url(address,limit=5000)
        print(x)
        ip='50.228.141.101:80'
        x=rq.get(x,timeout=timeout)
        try:

            x=json.loads(x.content)
            if(x['n_tx']<5000):
                x=df(x['txs'])
                return x
            else:

                print('超過5000筆')
        except:
            print(x.content)
    except rq.exceptions.Timeout:
        print(f'請求超時，等待時間超過 {timeout} 秒')
def oversold(buy):
    oversold_zone=0
    for i in buy.rsi:
        if i<30:
            oversold_zone+=1
    oversold_zone_ratio=oversold_zone/len(buy)
    return oversold_zone_ratio

def overbought(sell):
    oversold_zone=0
    for i in buy.rsi:
        if i>70:
            oversold_zone+=1
    oversold_zone_ratio=oversold_zone/len(buy)
    return oversold_zone_ratio

def frequency(x):
    
    return x.time.diff().mean()*-1

def loc_time(time,div):
    time=int(time)
    t=str(np.datetime64(time-time%div,'s'))
    return t


In [775]:
def oversold(buy,zone=30):
    oversold_zone=0
    for i in buy.rsi:
        if i<zone:
            oversold_zone+=1
    oversold_zone_ratio=oversold_zone/len(buy)
    return oversold_zone_ratio

def overbought(sell,zone=70):
    oversold_zone=0
    for i in sell.rsi:
        if i>zone:
            oversold_zone+=1
    oversold_zone_ratio=oversold_zone/len(buy)
    return oversold_zone_ratio

In [1141]:
def extract_address(address):
    x=get_data(address)
    address_list=[]
    for i in range(len(x)):
        y=(x['inputs'][i][0]['prev_out']['addr'])
        if y not in address_list:
            address_list.append(y)
    return address_list
def rsi_index(rsi,address):
    #4H
    try:

        rsi_sell=pd.DataFrame(columns=['rsi', 'time'])
        rsi_buy=pd.DataFrame(columns=['rsi', 'time'])
        for i in range(len(address)):
            if address.result[i]<0:
                #代表賣出

                x=address.time[i]
                x=int(x)
                x=np.datetime64(x-x%14400,'s')
                
                rsi_sell.loc[i]={'rsi':rsi.loc[str(x)],'time':x}
            elif address.result[i]>0:
                x=address.time[i]
                x=int(x)
                x=np.datetime64(x-x%14400,'s')
                
                rsi_buy.loc[i]={'rsi':rsi.loc[str(x)],'time':x}
        print(f'平均買進：{rsi_buy.rsi.mean()}\n平均賣出：{rsi_sell.rsi.mean()}')
        return rsi_buy,rsi_sell
    except:
        print('index不符')
def rsi_index_V2(rsi,address):
    #4H
    try:

        rsi_sell=pd.DataFrame(columns=['rsi', 'time','value'])
        rsi_buy=pd.DataFrame(columns=['rsi', 'time','value'])
        
        for i in range(len(address)):
            if address.result[i]<0:
                #代表賣出

                x=address.time[i]
                x=int(x)
                x=np.datetime64(x-x%14400,'s')
                
                rsi_sell.loc[i]={'rsi':rsi.loc[str(x)],'time':x,'value':address.result[i]}
                

            elif address.result[i]>0:
                x=address.time[i]
                x=int(x)
                x=np.datetime64(x-x%14400,'s')
                rsi_buy.loc[i]={'rsi':rsi.loc[str(x)],'time':x,'value':address.result[i]}
                
        B,S=rsi_weight_average(rsi_buy),rsi_weight_average(rsi_sell)
        print(f'加權平均買進：{B}\n加權平均賣出：{S}')
        return B,S,
    except:
        print('index不符')
def address_value(address):
    try:

        buy_value=[]
        sell_value=[]
        for i in range(len(address)):
            if address.result[i]<0:
               
                
                sell_value.append(-address.result[i])

            elif address.result[i]>0:
               
                buy_value.append(address.result[i])
        
        
        
        return buy_value,sell_value
    except:
        print('index不符')
def total_value(address):
    buy_total = sum(i for i in address.result if i > 0)  # 比较数字而不是字符串
    sell_total = sum(i for i in address.result if i < 0)  # 比较数字而不是字符串
    return buy_total, sell_total


In [1202]:
def count_BandS(address):
    B=(address['result']>0).sum()
    S=(address['result']<0).sum()
    return B,S


In [1208]:
target.to_csv('result0304.csv')

In [1205]:


for i in range(len(target)):
    
    address=target.address[i]
    path=path=f"data/{address}.csv"
    
    try:
        x=pd.read_csv(path)
        
        target.buying_time[i],target.selling_time[i]=count_BandS(x)
        print(target.buying_time[i],target.selling_time[i])
    except FileNotFoundError as e:
        print(f"找不到檔案：{path}")
        # 在這裡設定 target.total_return_ratio[i] 為預設值，或者其他適當的處理邏輯
        # target.total_return_ratio[i] = None  # 設定為預設值，或其他適當的值
        print("設定預設值並繼續下一個迴圈")
    except Exception as e:
        print(f"發生未知錯誤：{e}")
        # 在這裡可以添加其他適當的處理邏輯


24 23
95 84
106 336
25 25
145 143
43 42
13 12
3 3
1 2
205 205
4 6
2 2
找不到檔案：data/3MrwhYwABPbkXjshBePXFvr2wt1pkF7JQS.csv
設定預設值並繼續下一個迴圈
找不到檔案：data/bc1qzmcqhu7zvu9n04et8nuslzxqevt2955w05xh87.csv
設定預設值並繼續下一個迴圈
找不到檔案：data/3PqgR5hdhwoMxF8r66ZmKsb7vDtLH7RpGD.csv
設定預設值並繼續下一個迴圈
找不到檔案：data/37xovArMq3mqBFrmDRWpDDJNhnVKgMDTdk.csv
設定預設值並繼續下一個迴圈
找不到檔案：data/bc1qlyh9nn4npnglak747kfjpqwqqrnftr0my8z8lm.csv
設定預設值並繼續下一個迴圈
找不到檔案：data/1A3eQ2Ha7vp1q5Sqy8kZyqsFz9ugPNDs2j.csv
設定預設值並繼續下一個迴圈
31 31
找不到檔案：data/1zAmiiX89mzqxAJ2Qoe1DN5S3sYT7PVzA.csv
設定預設值並繼續下一個迴圈
92 90
2523 1030
113 111
19 19
16 13
65 60
3 3
2 2
88 85
1 1
49 49
3 3
7 7
66 62
1 1
6 5
8 8
24 24
199 193
939 699
27 27
126 124
找不到檔案：data/3AWGaggM87Nnhit5DwYYfXyvzW1NYutCw2.csv
設定預設值並繼續下一個迴圈
15 10
17 17
176 159
243 237
41 39
423 381
68 68
82 74
68 66
10 10
241 189
498 482
3 3
52 51
5 5
39 39
34 31
172 157
63 60
13 12
2 2
21 21
24 23
129 118
323 221
103 59
537 479
360 281
1 1
34 32
1 1
134 128
40 36
147 128
494 274
58 55
58 53
找不到檔案：data/16N3rbebUAeStsyTkj6

In [ ]:
for i in range(len(target)):
 
    address=target.address[i]
    path=path=f"data/{address}.csv"
    
    try:
        x=pd.read_csv(path)
        
       
        target.buy_rsi6[i],target.sell_rsi6[i]=rsi_index_V2(rsi6,x)
        target.buy_rsi12[i],target.sell_rsi12[i]=rsi_index_V2(rsi12,x)
        target.buy_rsi24[i],target.sell_rsi24[i]=rsi_index_V2(rsi24,x)
        target.buy_rsi48[i],target.sell_rsi48[i]=rsi_index_V2(rsi48,x)
        target.buy_rsi96[i],target.sell_rsi96[i]=rsi_index_V2(rsi96,x)
        target.buy_value[i],target.sell_value[i]=total_value(x)
        print(i)

    except FileNotFoundError as e:
        print(f"找不到檔案：{path}")
        # 在這裡設定 target.total_return_ratio[i] 為預設值，或者其他適當的處理邏輯
        # target.total_return_ratio[i] = None  # 設定為預設值，或其他適當的值
        print("設定預設值並繼續下一個迴圈")
    except Exception as e:
        print(f"發生未知錯誤：{e}")
        # 在這裡可以添加其他適當的處理邏輯

In [1184]:
result = target['buy_rsi12'] * target['buy_value']
print(f'buy_rsi12平均：{result.sum()/target.buy_value.sum()}')

result=target.buy_rsi24*target.buy_value
print(f'buy_rsi24平均：{result.sum()/target.buy_value.sum()}')
result=target.buy_rsi48*target.buy_value
print(f'buy_rsi48平均：{result.sum()/target.buy_value.sum()}')
result=target.buy_rsi96*target.buy_value
print(f'buy_rsi96平均：{result.sum()/target.buy_value.sum()}')
result=target.buy_rsi6*target.buy_value
print(f'buy_rsi6平均：{result.sum()/target.buy_value.sum()}')

buy_rsi12平均：53.26136520148548
buy_rsi24平均：53.112127020615624
buy_rsi48平均：52.81288987614463
buy_rsi96平均：52.62381376094834
buy_rsi6平均：53.41637302784058


In [1090]:
def rsi_weight_average(rsi_buy):
    numerator=sum(rsi_buy['rsi']*rsi_buy['value'])
    denominator=sum(rsi_buy['value'])
    return numerator/denominator

def sum_weight(rsi_buy,n=None,):
    
    
        

    if n ==0:
        
        return 0

    elif n==1:
        return rsi_buy.loc[0]['rsi']*rsi_buy.loc[0]['value']*(sum(rsi_buy.loc[:len(rsi_buy)]['value']))
        
    
  
    else:

        numerator=rsi_buy.loc[n-1]['rsi']*rsi_buy.loc[n-1]['value']+rsi_weight_average(rsi_buy,n-1)
        return numerator/sum(rsi_buy.loc[:len(rsi_buy)]['value'])
        
   
        
    
rsi_buy=pd.DataFrame(columns=['rsi','value'])
rsi_buy['rsi']=1,2,1
rsi_buy['value']=1,5,1



In [1188]:
address=target.address[3]


path=path=f"data/{address}.csv"
x=pd.read_csv(path)

In [1091]:
rsi_weight_average(rsi_buy)



1.7142857142857142

In [ ]:
def IRR(x):
    
    amount=[]
    price=[]

    open=ohlcv.open
    amount.extend(x.result)
    price.extend(x.time)
    for i in range(len(price)):
        price[i]=price[i]-(price[i]%60)
        price[i]=open.loc[str(np.datetime64(price[i],'s'))]

    amount = [p * -a for p, a in zip(price, amount)]

    import numpy_financial as npf

    irr = npf.irr(amount)
    print(f'Internal Rate of Return (IRR): {irr:.2%}')
    return irr

In [ ]:
reloaded_module = importlib.reload(__import__('total_return'))


In [ ]:
import total_return as t
import importlib
reloaded_module = importlib.reload(__import__('total_return'))



In [ ]:
t.calulate_total_return_ratio(x)

In [ ]:
for i in range(len(target)):
    
    address=target.address[i]
    path=path=f"data/{address}.csv"
    
    try:
        x=pd.read_csv(path)
        
        buy6,sell6=rsi_index(rsi6,x)
        print(i)
        buy12,sell12=rsi_index(rsi12,x)
        target.buy_rsi12[i],target.sell_rsi12[i]=buy12.rsi.mean(),sell12.rsi.mean()
        print(i)

    except FileNotFoundError as e:
        print(f"找不到檔案：{path}")
        # 在這裡設定 target.total_return_ratio[i] 為預設值，或者其他適當的處理邏輯
        # target.total_return_ratio[i] = None  # 設定為預設值，或其他適當的值
        print("設定預設值並繼續下一個迴圈")
    except Exception as e:
        print(f"發生未知錯誤：{e}")
        # 在這裡可以添加其他適當的處理邏輯

In [782]:

count=2
# count=76

# +=1

address=target.address[count]

x=get_data(address)
path=path=f"data/{address}.csv"
# x=pd.read_csv(path)

target.交易次數[count]=len(x)

target.IRR[count]=IRR(x)
buy,sell=rsi_index(rsi48,x)
target.buy_rsi48[count],target.sell_rsi48[count]=buy.rsi.mean(),sell.rsi.mean()


target.oversold_zone_ratio48[count]=oversold(buy)
target.overbought_zone_ratio48[count]=overbought(sell)
print(f'超買區{overbought(sell)}')

buy,sell=rsi_index(rsi96,x)
target.buy_rsi96[count],target.sell_rsi96[count]=buy.rsi.mean(),sell.rsi.mean()

target.oversold_zone_ratio96[count]=oversold(buy)
target.overbought_zone_ratio96[count]=overbought(sell)

buy,sell=rsi_index(rsi24,x)
target.buy_rsi24[count],target.sell_rsi24[count]=buy.rsi.mean(),sell.rsi.mean()
target.oversold_zone_ratio24[count]=oversold(buy)

target.overbought_zone_ratio24[count]=overbought(sell)



total=t.calulate_total_return_ratio(x,open=ohlcv.open)
print(total)
target.total_return_ratio[count]=total
path=path=f"data/{address}.csv"
x.to_csv(path)

target.交易頻率[count]=frequency(x)
print(count)

https://blockchain.info/rawaddr/bc1qepz9ymljjdp4afahzmlt3w706pvwxc4uva5ha6?limit=5000&offset=0
Internal Rate of Return (IRR): -0.27%
平均買進：55.12616883749969
平均賣出：52.78834858608881
超買區0.05660377358490566
平均買進：53.342476581717406
平均賣出：52.432518106116945
平均買進：56.255058324122274
平均賣出：52.875242779689685
2.245883497737967
2


In [ ]:
path=path=f"data/{address}.csv"
path

In [745]:
target.to_csv('/Users/caibinghong/cryptofinlab/onchain/target0205V2.csv')

In [ ]:
# address="19sQKAvLTNpnBDFu58eapDUiHZLfb6tRvN"
def check_and_added_address(address_list,target):
    for address in  address_list:

        if address not in target['address'].values:
            df_addr = pd.DataFrame({'address': [address]})
            target = pd.concat([target, df_addr], ignore_index=True)
            print('加入成功')
        else:
            print('重複')
    return target

for address in  address_list:

        if address not in target['address'].values:
            df_addr = pd.DataFrame({'address': [address]})
            target = pd.concat([target, df_addr], ignore_index=True)
            print('加入成功')
        else:
            print('重複')


address='1E6G367pe9g4fh1kbfCJqbjC2v3BaPK3yT'
if address not in target['address'].values:
        df_addr = pd.DataFrame({'address': [address]})
        target = pd.concat([target, df_addr], ignore_index=True)
        print('加入成功')
else:
        print('重複')


In [953]:
buy,sell=rsi_index(rsi6,x)
target.buy_rsi6[0],target.sell_rsi6[0]=buy.rsi.mean(),sell.rsi.mean()
buy,sell=rsi_index(rsi12,x)
target.buy_rsi12[0],target.sell_rsi12[0]=buy.rsi.mean(),sell.rsi.mean()
buy,sell=rsi_index(rsi24,x)
target.buy_rsi24[0],target.sell_rsi24[0]=buy.rsi.mean(),sell.rsi.mean()

平均買進：56.37859089257753
平均賣出：52.56534152720477
平均買進：56.540884822366856
平均賣出：52.77904948065374
平均買進：56.255058324122274
平均賣出：52.875242779689685


In [ ]:

target
buy,sell=rsi_index(rsi6,x)
target.buy_rsi6[1],target.sell_rsi6[1]=buy.rsi.mean(),sell.rsi.mean()
target.oversold_zone_ratio[1]=oversold(buy)
target.overbought_zone_ratio[1]=overbought(sell)


In [ ]:
# z+=1
address=research_data.address[z]
x=get_data(address)
research_data.交易次數[z]=len(x)
print(f'交易次數:{len(x)}')
research_data.交易頻率[z]=x.time.diff().mean()*-1


import talib
rsi=talib.RSI(ohlcv_4h.close,timeperiod=96)
buy,sell=rsi_index(rsi,x)
research_data.buy_RSI_4h[z],research_data.sell_RSI_4h[z]=buy.rsi.mean(),sell.rsi.mean()
# rsi5m=talib.RSI(ohlcv_5m.close,timeperiod=96)
# buy,sell=rsi_index(rsi5m,x)
research_data.buy_RSI_5m[z],research_data.sell_RSI_5m[z]=buy.rsi.mean(),sell.rsi.mean()

amount=[]
price=[]
key=x.time[0]
mo=60*60*24*30
key=key-mo
open=ohlcv.open
amount.extend(x.loc[x['time'] > key, 'result'])
price.extend(x.loc[x['time'] > key, 'time'])
for i in range(len(price)):
    price[i]=price[i]-(price[i]%60)
    price[i]=open.loc[str(np.datetime64(price[i],'s'))]

amount = [p * -a for p, a in zip(price, amount)]

import numpy_financial as npf

irr = npf.irr(amount)
print(f'Internal Rate of Return (IRR): {irr:.2%}')

research_data.月報酬率[z]=irr

random_wait_time = random.uniform(1, 5)
# 打印等候时间
print(f"等候时间：{random_wait_time:.2f} 秒")
# 等待随机时间
time.sleep(random_wait_time)
    

In [ ]:
print(
research_data.sell_RSI_4h.mean(),
research_data.sell_RSI_4h.median())

In [ ]:
research_data.to_csv('research_data2.csv')